<a href="https://colab.research.google.com/github/DWGodwin/Adleo_final/blob/main/notebooks/cGAN_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using a Conditional GAN (cGAN) to Generate Synthetic Data for Training of Semantic Segmentation Models

# 1. Introduction

We undertook reimplementation of a conditional Generative Adversarial Network (cGAN) that was originally developed by (https://github.com/gbaier/rs_img_synth). Our aim was to generate synthetic Planetscope images, and to achieve this, we utilized a cGAN architecture that consists of two neural networks: a generator and a discriminator, which are trained in an adversarial manner. 

We created a custom dataset module that could read our dataset. We also made modifications to the gan.py module so that it could load our dataset and feed it into the generator and discriminator for training/testing the cGAN.

We conducted several experiments to improve the quality of the generated synthetic images. Here is a summary of the experiments we performed: 

1. Normalizing each individual DEM between 0 and 1.
2. Normalizing the DEM between 0 and 1 across the range of the entire dataset. This was done to ensure that elevation information was kept relative to other images.
3. Normalizing the DEM between 0 and 1 for each DEM tile and adding a gaussian noise vector that was also normalized between 0 and 1.
4. Normalizing the DEM between 0 and 1 across the range of the dataset,and adding a gaussian noise vector between 0 and 1
5. Replacing the DEM entirely with a Gaussian noise vector that was normalized between 0 and 1.

Prior to conducting these experiments, we initially used the original DEM values as input with and without adding the noise, but, the results were not satisfactory. As a result, we excluded these findings from our final results.
(ra)




## 2. Methods


### 2.1 Data

Our study area is Republic of Congo and the dataset comprised of 435 image and label pairs obtained from PlanetScope imageries. The labels were digitized manually using QGIS tools and then rasterized into 200 by 200 pixels. As the downloaded images were not of the same size as the labels, we utilized the chipping code to chip the images to match the extent of their corresponding labels. Additionally, using ee_downloader code, we downloaded NASA SRTM Digital Elevation 30m and resampled them to 2.5-meter resolution, similar to the resolution of the other inputs.(ra)

### 2.2 Model Details


#### 2.2.0 Code structure overview

the generator takes as inputs a DEM raster layer and an abstract land-cover classification layer, where land cover is separated into three classes of field, not field and field boundary.

The raster layer is fed through the encoder and decoder of the generator architecture, while the labels layer is fed into each layer using SPADE normalization after the regular batch norm of that layer. This process outputs a 4-channel synthetic image.

This image and the label raster are fed into the discriminator, which samples the image at 3 scales and outputs an average of the discriminator scores for each scale.

Once the DEMs, labels, and rasters are preprocessed and loaded into a training DataLoader object and a validation DataLoader object, we run the training loop using this dataset.(dg)

#### 2.2.1 Initializing Training
When train.py is run with the given parameters, the program first imports the necessary modules from the repository, and uses the get_parser() function within the options.gan module to create an object of the parameters to feed into later modules. This is stored within args, i.e. args.seed uses the key "seed" to retreive the seed value if we have set one in initializing train.py

train.py then initializes the logger and checks the command line arguments:

- checks whether we have defined a seed for generating the latent vector, a feature which is included for reproducability purposes. Running the training using the same seed should produce deterministic results, allowing other researchers to reproduce our work.

- checks the length of crop and resize parameters, and sets these arguments equal to the first argument within them.

- creates a dictionary of the parameters to save into the .yaml, named "CONFIG"

The code then creates train_transforms and test_transforms using options.common.get_transforms(CONFIG), calling the dicitonary of parameters.

The code then configures the dataset and neural network for training using options.common.get_dataset(), options.gan.get_generator(), and options.gan.get_discriminator(). 

Then, it sets up distributed training environment for the generator and discriminator using torch.nn.parallel.DistributedDataParallel().

Then, it trains the model using Trainer.train().(dg)


#### 2.2.2 Getting the Dataset

Within train.py using option.common.get_dataset():

get_transforms(config): This function returns the train and test transforms for the dataset. It takes a configuration dictionary as input, which is generated by args2dict. The transforms include random cropping, horizontal flipping, resizing, and tensor conversion. The transforms depend upon the dataset parameter defined when initializing train.py.

This code allows us to define how we transform a given dataset, with different transforms for different data. All transforms are functions within the datasets.transforms.py module.

get_dataset(config, split, transforms): This function returns a dataset class. It takes a configuration dictionary, a string representing the dataset split (i.e., train or test), and the train or test transforms generated by get_transforms as inputs. It selects the dataset based on the configuration and returns the dataset class.

get_dataset() uses config to define where to look for the dataset. The name of the dataset is taken as a command to run that dataset's module. The dataset modules are named as [dataset name].py, in our case drc.py. This function looks in the root directory of the dataset and iterates over the catalog CSV we provide in order to return the dataset class with transforms to be passed through get_dataset on to the training function.(dg)

####2.2.3 Setting Up the Generator and Discriminator

Now that we have the correct dataset class, we run options.gan.get_generator() and options.gan.get_discriminator() in order to set up the generator and discriminator models.

get_generator() first retrieves the number of labels and output channels from the dataset class based on the configuration. It then checks if the input to the generator is only the segmentation map. If so, it returns a SPADE generator. Otherwise, it checks if there is no segmentation map as input. In this case, it returns a ResNet encoder decoder model.

As we are using "seg" in our inputs, get_generator will use models.generator.SPADEGenerator() to create our generator architecture.

get_discriminator() always returns models.discriminator.Multiscale(d_nets), in which d_nets is a list of models.discriminator.PatchGAN() using number of classes and number of scales as defined in the model config object.

Back within train.py, we create these models and send them to CUDA with:

```
g_net = options.gan.get_generator(CONFIG).to(device)
d_net = options.gan.get_discriminator(CONFIG).to(device)
```

(dg)

#### 2.2.4 The Model Architectures

2.2.4.1. The gerenator architecture:

Our generator takes as inputs a DEM raster layer, an abstract segmentaion map label (with 2 class: field and non-field), and a noise vector, then it outputs a fake image.

The generator has 2 main components: the encoder and decoder. Each block in the encoder is a convolution block which downsamples the images to extract spatial feature. Each block in the decoder is either an upsampling layer (nearest neighbor) or a ResNet block to overcome checkerboard artifacts.

The raster layer (DEM) is fed through the encoder and decoder of the generator architecture, while the label layer is fed into each layer using SPADE normalization after the regular batch norm of that layer. The goal of the SPADE normalization is for the model to learn the affine transformation of the land cover. This process outputs a 4-channel synthetic image.(nh)


2.2.4.2. The discriminator architecture:

PatchGAN class:

We set the discriminator to takes a 4-channel input and have 4+1 layers. The first layer is an initial convolutional layer with 4 input channels and 64 output channels. It uses a 4x4 kernel with a stride of 2 and a LeakyReLU activation with a negative slope of 0.2.
The subsequent layers each consist of a convolutional layer followed by an [InstanceNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.InstanceNorm2d.html) layer and a LeakyReLU activation. The number of input channels for each layer is doubled compared to the previous layer, starting at 64 and increasing to 256. The kernel size is 4x4 and the stride is 2 for the first three layers, and 1 for the final layer. Each convolutional layer is also passed through [spectral normalization](https://pytorch.org/docs/stable/generated/torch.nn.utils.spectral_norm.html). All layers set to have the same padding.

Finally, there is a final convolutional layer with 512 input channels and 1 output channel, which also uses a 4x4 kernel and stride of 1. The output is a scalar value, representing the discriminator's confidence in whether the input is real or fake.

The discriminator returns both the final output and a list of intermediate feature maps, which can be used for feature matching loss.(ra)


In [ ]:
#@title 1
# Number of channels for each layer based on input number of channels and the number of layers: 
input_nc = 4
init_nc = 64
n_layer = 4+1
ncs = [init_nc * 2 ** n_layer for n_layer in range(n_layer - 1)]
print(f"The first layer has been set to have {input_nc}, the second layer has {ncs[0]}, the third layer has {ncs[1]}, and the fouirth layer has {ncs[2]}, and the final layer has {ncs[-1]} channels.")

The first layer has been set to have 4, the second layer has 64, the third layer has 128, and the fouirth layer has 256, and the final layer has 512 channels.


In [ ]:
#@title 2

# Stride in each layer:
strides = (len(ncs) - 1) * [2] + [1]
print(f"The second layer is using stride {strides[0]}, the third layer is using stride {strides[1]}, and the last layer is using stride {strides[-1]}.")

The second layer is using stride 2, the third layer is using stride 2, and the last layer is using stride 1.


Multiscale class: 

The PatchGAN class defines a single discriminator network that operates at a fixed scale and is used to classify images as real or fake. 
The Multiscale class, then defines a discriminator network that operates at multiple scales. It is composed of multiple instances of the PatchGAN class, with each instance operating at a different scale.
In the Multiscale class, the outputs of each PatchGAN discriminator are concatenated and flattened to produce a final output. This final output is used to compute the adversarial loss for the generator. 
The number of discriminators can be set as a hyperparameter. (ra)


#### 2.2.5 The Training Loop

Once the arguments are passed to all necessary modules and the generator and discriminator are initialized, train.py defines a trainer as 

```
trainer = Trainer(
    g_net,
    d_net,
    args.input,
    args.output,
    feat_loss=CONFIG["training"]["lbda"],
)
```

Using the Trainer class we imported in ```from trainer import Trainer```. 
We then define a train_sampler and pass this to train_dataloader, the dataloader object we create using the dataset class defined in 2.2.2

Finally, we are ready to train. We pass the Trainer class to train using the train_dataloader for args.epochs. This runs the trainer.py code under Trainer.train(self, dataloader, n_epochs).

Trainer.train() calculates a running loss for both the generator and discriminator for every index and sample of enumerate(dataloader).

Train.py then saves the generator and discriminator state dictionaries if args.local_rank == 0
(dg)

## 3. Challenges

To successfully run the source repository, we faced several challenges that needed to be addressed. First, the initialization modules were not defined in the repository, so we had to add them. Additionally, there were several typos in the code that needed to be fixed.

We also encountered issues with using the torch.distributed function for parallelizing the process, so we decided to eliminate that part. Furthermore, we needed to adjust either the generator's encoder/decoder number of layers or our input sizes to ensure that the chip sizes were divisible by the number of encoder/decoder layers. To address this, we cropped our input images to 192 by 192.


Another challenge we encountered was that our DEM layer did not contain sufficient information for the generator to utilize its distribution in generating synthetic images. The limited information was partly due to our resampling of the original 30-meter DEM resolution to 2.5 meters, and also because the elevation of the country was relatively uniform, particularly in areas with agricultural fields. To overcome this limitation, we conducted several experiments to improve the quality of the generated images. (ra)

## 4. Results

### 3.1 Results of Experiment 1:

Random cropping to 192, random horizontal flip, random vertical flip, each DEM normalized between 0 and 1

epoch  80, g_loss:  4.676, d_loss:  3.747

**Progress of generator during training** 
<table>
  <tr>
    <th>Gif of all generated images from epoch 2</th>
    <th>Gif of all generated images from epoch 80</th>
  </tr>
  <tr>
    <td><img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/generated_epoch_2_self_norm.gif?raw=true" loop></td>
    <td><img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/generated_epoch_80_self_norm.gif?raw=true" loop></td>
  </tr>
</table>

**Results of test**

Each column of the table contains one sample of the test dataset. The model has never seen these DEMs or labels before.

From top to bottom: DEM with added noise vector, label showing edges of crop fields, generated image, ground truth image

<img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/model_test_self_norm.jpg?raw=true">

### 3.2 Results of Experiment 2:

Random cropping to 192, random horizontal flip, random vertical flip, normalizing the DEM between 0 and 1 across the range of the dataset

epoch  80, g_loss:  4.126, d_loss:  3.872

**Progress of generator during training** 
<table>
  <tr>
    <th>Gif of all generated images from epoch 2</th>
    <th>Gif of all generated images from epoch 80</th>
  </tr>
  <tr>
    <td><img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/generated_epoch_2_all_norm.gif?raw=true" loop></td>
    <td><img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/generated_epoch_80_all_norm.gif?raw=true" loop></td>
  </tr>
</table>

**Results of test**

Each column of the table contains one sample of the test dataset. The model has never seen these DEMs or labels before.

From top to bottom: DEM with added noise vector, label showing edges of crop fields, generated image, ground truth image

<img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/model_test_all_norm.jpg?raw=true">

### 3.3 Results of Experiment 3:

Random cropping to 192, random horizontal flip, random vertical flip, normalizing the DEM between 0 and 1 for each DEM tile and adding a gaussian noise vector that was also normalized between 0 and 1.

epoch  80, g_loss:  4.713, d_loss:  3.811

**Progress of generator during training** 
<table>
  <tr>
    <th>Gif of all generated images from epoch 2</th>
    <th>Gif of all generated images from epoch 80</th>
  </tr>
  <tr>
    <td><img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/generated_epoch_2_self_norm_and_noise.gif?raw=true" loop></td>
    <td><img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/generated_epoch_80_self_norm_and_noise.gif?raw=true" loop></td>
  </tr>
</table>

**Results of test**

Each column of the table contains one sample of the test dataset. The model has never seen these DEMs or labels before.

From top to bottom: DEM with added noise vector, label showing edges of crop fields, generated image, ground truth image

<img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/model_test_self_norm_and_noise.jpg?raw=true">

### 3.4 Results of Experiment 4:

Random cropping to 192, random horizontal flip, random vertical flip, normalizing the DEM between 0 and 1 across the range of the dataset,and adding a gaussian noise vector between 0 and 1

epoch  80, g_loss:  4.126, d_loss:  3.872

**Progress of generator during training** 
<table>
  <tr>
    <th>Gif of all generated images from epoch 2</th>
    <th>Gif of all generated images from epoch 80</th>
  </tr>
  <tr>
    <td><img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/generated_epoch_2_transform_all_norm_add_noise.gif?raw=true" loop></td>
    <td><img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/generated_epoch_80_transform_all_norm_add_noise.gif?raw=true
" loop></td>
  </tr>
</table>

**Results of test**

Each column of the table contains one sample of the test dataset. The model has never seen these DEMs or labels before.

From top to bottom: DEM with added noise vector, label showing edges of crop fields, generated image, ground truth image

<img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/model_test_transform_all_norm_add_noise_80_epochs.jpg?raw=true">

### 3.5 Results of Experiment 5:

Random cropping to 192, random horizontal flip, random vertical flip, replacing the DEM with a gaussian noise vector between 0 and 1.

epoch  80, g_loss:  8.498, d_loss:  3.567

**Progress of generator during training** 
<table>
  <tr>
    <th>Gif of all generated images from epoch 2</th>
    <th>Gif of all generated images from epoch 80</th>
  </tr>
  <tr>
    <td><img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/generated_epoch_2_transform_only_noise.gif?raw=true" loop></td>
    <td><img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/generated_epoch_80_transform_only_noise.gif?raw=true
" loop></td>
  </tr>
</table>

**Results of test**

Each column of the table contains one sample of the test dataset. The model has never seen these DEMs or labels before.

From top to bottom: DEM with added noise vector, label showing edges of crop fields, generated image, ground truth image

<img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/model_test_transform_noise_only.jpg?raw=true">

(dg)

## 5. Discussion

### 5.1 Comparison of Experiments


All experiments showed similar losses after 80 epochs of training with all the same hyperparameters and random transformations, with the only difference being the normalization of the DEM layer and the inclusion or disinclusion of random noise and the DEM. Experiment 7 showed a higher generator loss, possibly due to the model being unable to memorize any patterns between the DEM and the label vector.(dg)

### 5.2 Effects of Noise and Normalization


The test results of the examples show a range of results, showing how changing noise and normalization in the condition being fed into the mode changes how it processes inputs

Experiment 2 shows the worst generalizability to the test set. While in the .gif of the last epoch of training, all images are within the range of plausability, and show tonal variability without being clear copies of input images. However, the outputs of the test set show that the model may be relying too much on the association of specific DEM elevations with corresponding labels. There is a variability in brightness in the dataset. It may be learning to associate certain label shapes with brightness, and other DEM elevations with brightness, and may create an image that is too bright when these DEMs and labels are presented together. This shows that the relationship between elevation and image on an absolute scale may not be robust.

Experiment 1 shows images that begin to replicate the texture of the input images, showing the texture of trees outside crop fields. There is some artifacting around some of the images, but are plausible other than that, although they generalize the area inside and outside crop field boundaries.

These results show that a conditional GAN does not necessarily benefit from the inclusion of a noise vector. It may be preferable to use randomized transformations instead, especially randomized cropping to small portions of input images. These will prevent the model memorizing inputs as keys to the desired outputs while maintaining the highest degree of informational richness in the inputs of the model.(dg)

### 5.3 Comparison to Methods and Results of Baier et al.


Our process for training the model most closely mirrored that of Baier et. al. in synthesizing RGB images from the GeoNRW dataset. This dataset consists of 1,000x1,000 RGB aerial photographs, high-resolution DEMS, and land cover masks with 10 classes. Some key differences required adaptation and may limit what we can expect from our model.(dg)

Here is an example of their inputs and outputs for comparison:

<img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/nrw_sar_rgb_comp.jpg?raw=true">
Source: Baier et. al, Building a Parallel Universe - Image Synthesis from Land Cover Maps and Auxiliary Raster Data

#### 5.3.1 Differences in RGB Inputs


The RGB images GeoNRW was feeding its model are high resolution and nearly 4x the size of their output image of 256x256. This allowed them to randomly crop 256x256 tiles many times from the same input image, randomizing the inputs effectively between epochs. This allowed them to introduce randomness without diluting their rich dataset with random noise vectors.

By contrast, we cropped our 200x200 tiles randomly to 192x192, increasing the risk that the model would memorize inputs as whole rather than weighting patterns in those inputs. This can be seen explicity in the results of experiments with no random noise vector - the model begins to reproduce training maps with great accuracy based on their input DEM and labels, placing non-farm clearings in places where the input does not necessarily suggest they would be. For example, from the experiment with normalized DEM and no noise vector:

**Inputs and Generated Image of Epoch 71, Batch 100, Experiment 3:**

<table>
  <tr>
    <td><img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/label_epoch_71_batch_100.jpg?raw=true"></td>
    <td><img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/dem_epoch_71_batch_100.jpg?raw=true"></td>
    <td><img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/truth_epoch_71_batch_100.jpg?raw=true"></td>
    <td><img src="https://github.com/DWGodwin/Adleo_final/blob/main/examples/generated_epoch_71_batch_100.jpg?raw=true"></td>
  </tr>
    <tr>
    <th>Label</th>
    <th>DEM</th>
    <th>Truth</th>
    <th>Generated</th>
  </tr>
</table>

Feeding the model uniform semantic information and a non-descriptive DEM, the model is starting to recall a specific image that it has been trained on. (dg)

#### 5.3.2 Differences in Semantic Inputs


While Baier et. al. use a 10-class semantic input, we define 3 classes for field, non-field, and field edge. The 10-class input allows the model to train weights for each of those classes and recreate their features when semantic maps are input.

Our the tests of the models shows that our 3 classes are sufficient to generate images which delineate between these classes generally. The model learns which rgb patterns are appropriate for each ot these classes. However, we sacrifice detail and texture within the non-field class, as rather than adding appropriate non-field features, the model returns the most general features for areas of each class.(dg)

#### 5.3.3 Differences in Elevation Inputs


Baier et. al. use high-resolution elevation information that captures textural information about the features they are attempting to train the model to recreate. This allows the model to learn to recreate texture and even shadows from elevation data - for example, in the second column of their table reproduced from their report, the taller buildings cast longer shadows than shorter buildings in the generated image. The combination of rich semantic and high-resolution elevation information is what allows their model to reproduce such convincing images, as well as their much larger quantity of data. The information from one is being used to evaluate how to interpret the other, and vice versa.

Our data is more limited. Not only was our DEM resampled, but also may not be as closely related to our other data as land cover and high resolution DEMS are - especially in our study area, which does not show any large changes in elevation within DEMs. Not only is it important to have high quality data, but also to feed the model inputs which are complementary in producing the output.(dg)

### 5.4 Further Recommendations

#### 5.4.1 Data

In continuing to adapt this model to future geospatial work, datasets with richer information and higher resolution should be selected.

In particular, the pairing of semantic information with high resolution DEMs allowed Baier et. al. to accurately replicate textural features of real images.

If no semantic information is available for a region, it could be synthesized. As we are not telling the model what each label signifies, we could segment an image into classes using unsupervised classification, add labels of interest, overwriting the segmented class at that location, and feed this into the model as a segmentation map. For example, if we had an unsupervised semantic segmentation map of the study area, it may create classes distinguishing grasslands, forest, and water. Then, it would be possible to feed the model this semantic map and train it to recognize patterns within these classes. 

After training, it would be possible to input even synthesized semantic maps not corresponding to a real place, along with any other condition it was trained on, to produce plausible false satellite imagery. Likewise, satellite imagery from other datasets could be used as a condition, as well as any other raster dataset that shows variation within a tile.

One possibility for expanding the model past its current state would be to modify it to accept and produce satellite imagery with a greater number of bands - limiting our model to RGB limits its utility in standard remote sensing analyis.(dg)

#### 5.4.2 Learning Rate Scheduling


One of the most pervasive issues in training our model was vanishing and exploding gradients. At the beginning of training, the learning rates recommended by Baier et. al. (0.0004 for the discriminator and 0.0001 for the generator) resulted in generator outputs that immediately went completely white, often followed by cycling through maximizing and minimizing the colors of the bands. Correspondingly, the discriminator's learning rate immediately dropped to zero, making it impossible for the GAN to progress.

Lowering the learning rate of both resulted in a stable learning progress. For our experiments, lowering the learning rate of the discriminator to 0.00001 and the learning rate of the generator to 0.00004 produced results consistently. However, training went slowly and may stall out once the model finds a strategy, especially as the discriminator learning rate is so low. A non-linear learning rate scheduler may be helpful to implement, starting the learning rate low to prevent exploding gradients, and increase it after the initial stage to speed training. (dg)
